In [2]:
import pandas as pd

# --- 1. Load the two CSV files ------------------------------------------------
# Adjust the paths if the CSVs are located elsewhere
spike_df  = pd.read_csv("Final Spike Predictions.csv",  parse_dates=["time"])
tgpt_df   = pd.read_csv("Final TimesGPT Data.csv",      parse_dates=["time"])

# --- 2. Merge on the shared “time” column ------------------------------------
# “inner” keeps only timestamps that appear in *both* files; switch to “outer”
# if you want a full union with NaNs where a file is missing a timestamp.
df = (
    pd.merge(
        spike_df,
        tgpt_df,
        how="inner",         # or "outer", "left", "right" as needed
        on="time",
        suffixes=("_spike", "_tgpt")   # helpful if the files share column names
    )
    .set_index("time")                 # make “time” the index
    .sort_index()                      # always nice to keep it chronological
)

# --- 3. (Optional) quick sanity checks ---------------------------------------
print(df.info())
print(df.head())


<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 8760 entries, 2023-01-01 00:00:00 to 2023-12-31 23:00:00
Data columns (total 10 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   pred_spike_prob  8760 non-null   float64
 1   pred_mag_MW      8760 non-null   float64
 2   true_spike       8760 non-null   int64  
 3   true_mag_MW      8760 non-null   float64
 4   Unnamed: 0       8760 non-null   int64  
 5   unique_id        8760 non-null   int64  
 6   DAP              8760 non-null   float64
 7   TimeGPT          8760 non-null   float64
 8   TimeGPT-hi-90    8760 non-null   float64
 9   TimeGPT-lo-90    8760 non-null   float64
dtypes: float64(7), int64(3)
memory usage: 752.8 KB
None
                     pred_spike_prob  pred_mag_MW  true_spike  true_mag_MW  \
time                                                                         
2023-01-01 00:00:00         0.000269    11.166585           0          0.0   
2023-01-01 01:

In [4]:
df = df.drop(['true_spike', 'true_mag_MW', 'Unnamed: 0', 'unique_id', 'TimeGPT-hi-90', 'TimeGPT-lo-90'], axis = 1)

In [6]:
df

,pred_spike_prob,pred_mag_MW,DAP,TimeGPT
time,,,,
2023-01-01 00:00:00,0.000269,11.166585,10.83340,0.794704
2023-01-01 01:00:00,0.000272,12.847394,10.22220,2.510757
2023-01-01 02:00:00,0.000188,12.488766,8.73719,-0.742447
2023-01-01 03:00:00,0.000166,11.972310,8.19749,2.853043
2023-01-01 04:00:00,0.000165,11.738667,8.33215,9.884354
...,...,...,...,...
2023-12-31 19:00:00,0.001989,9.345063,19.37150,16.540146
2023-12-31 20:00:00,0.001092,9.124728,16.77390,13.965488
2023-12-31 21:00:00,0.000679,9.029976,15.00350,12.675762


In [12]:
import numpy as np

SPIKE_THRESHOLD = 0.0          # e.g. set to 0.7, 0.5, etc.

df["final_pred"] = np.where(
    df["pred_spike_prob"] > SPIKE_THRESHOLD,
    (1 - df["pred_spike_prob"]) * df["TimeGPT"]
    + df["pred_spike_prob"]   * df["pred_mag_MW"],
    df["TimeGPT"]
)

df


,pred_spike_prob,pred_mag_MW,DAP,TimeGPT,final_pred
time,,,,,
2023-01-01 00:00:00,0.000269,11.166585,10.83340,0.794704,0.797496
2023-01-01 01:00:00,0.000272,12.847394,10.22220,2.510757,2.513569
2023-01-01 02:00:00,0.000188,12.488766,8.73719,-0.742447,-0.739957
2023-01-01 03:00:00,0.000166,11.972310,8.19749,2.853043,2.854557
2023-01-01 04:00:00,0.000165,11.738667,8.33215,9.884354,9.884660
...,...,...,...,...,...
2023-12-31 19:00:00,0.001989,9.345063,19.37150,16.540146,16.525833
2023-12-31 20:00:00,0.001092,9.124728,16.77390,13.965488,13.960204
2023-12-31 21:00:00,0.000679,9.029976,15.00350,12.675762,12.673286


In [13]:
import numpy as np
from sklearn.metrics import mean_absolute_error 

mae = mean_absolute_error(df["DAP"], df["final_pred"])
print(f"MAE for {df.index.year.unique().tolist()} : {mae:,.3f} $/MWh")


MAE for [2023] : 20.582 $/MWh


In [14]:
from sklearn.metrics import mean_absolute_error

AUG_YEAR = 2023   

aug_mask = df.index.month == 8
if AUG_YEAR is not None:
    aug_mask &= df.index.year == AUG_YEAR

aug_df = df.loc[aug_mask]

# Guard-rail: check that we actually captured data
if aug_df.empty:
    raise ValueError(f"No August data found for year = {AUG_YEAR}")

# Compute MAE
mae_august = mean_absolute_error(aug_df["DAP"], aug_df["final_pred"])

print(f"MAE for August{'' if AUG_YEAR is None else ' ' + str(AUG_YEAR)}: {mae_august:,.3f} $/MWh")


MAE for August 2023: 95.107 $/MWh
